In [1]:
!pip install py2neo
!pip install neo4j
!pip install spacy
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 5.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 173 kB 5.2 MB/s 
  Created wheel for neo4j: filename=neo4j-5.1.0-py3-none-any.whl size=248097 sha256=78e78d4dc23670a555be34047665d0b9a926e75d268f923b1509904f26a6dbcc
  Stored in directory: /root/.cache/pip/wheels/64/96/da/c30ec8577a970eadecc436103fbcbf7af1d6398d0ebe32cbab
Successfully built neo4j
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.0 MB/s 
     |████████████████████████████████| 71 kB 4.6 MB/s 
     |████████████████████████████████| 13.5 MB 45.7 MB/s 
     |████████████████████████████████

In [14]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.9 MB 1.3 MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.5.1-py3-none-any.whl size=15870867 sha256=2b64fdcf50c3100502eec0dd74113109815287f6f1796004188c0ecf5d8f852f
  Stored in directory: /root/.cache/pip/wheels/cb/0d/9c/03895763b9041405073120fbd58489e4f0d8da9c6bff441b05
Successfully built en-core-sci-sm


In [12]:
import requests
import json
import csv
import io
import spacy
from py2neo import Graph
from neo4j import GraphDatabase, basic_auth

In [3]:
def SubmitPMIDList(Pmid,Format,Bioconcept):

    # json = {"Pmids": Pmid}

    json = {}

    #
    # load pmids
    #
    with io.open(Inputfile,'r',encoding="utf-8") as file_input:
        json = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
    if Bioconcept != "": 
        json["concepts"] = Bioconcept.split(",")
    
    print(json)
    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/" + Format , json = json)
    
    res = r.json()
    print(res)

    pmid = res['id']
    res0 = res['passages'][0]['annotations']
    res1 = res['passages'][1]['annotations']
    
    result = {}
    result = res0 + res1
    
    for i in result:
        i['Pmid'] = Pmid
    return result
    
    if r.status_code != 200 :
        print ("[Error]: HTTP code "+ str(r.status_code))
    else:
        return result

In [7]:
Inputfile = "./pmid"
Format = "biocjson"
Bioconcept = ""
res_json=[]

In [8]:
with io.open(Inputfile,'r',encoding="utf-8") as file_input:
    pmidlist = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
for i in pmidlist['pmids']:
    res_json.extend(SubmitPMIDList([i], Format, Bioconcept))

{'pmids': ['35900868']}
{'_id': '35900868|None', 'id': '35900868', 'infons': {}, 'passages': [{'infons': {'journal': 'J Clin Invest;2022Jul28. doi:10.1172/JCI161908', 'year': '2022', 'type': 'title', 'authors': 'Lone MA, Aaltonen MJ, Zidell A, Pedro HF, Morales Saute JA, Mathew S, Mohassel P, Bonnemann CG, Shoubridge EA, Hornemann T, ', 'section': 'Title'}, 'offset': 0, 'text': 'SPTLC1 variants associated with ALS produce distinct sphingolipid signatures through impaired interaction with ORMDL proteins.', 'sentences': [], 'annotations': [{'id': '2', 'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'}, 'text': 'SPTLC1', 'locations': [{'offset': 0, 'length': 6}]}, {'id': '3', 'infons': {'identifier': 'MESH:D013107', 'type': 'Chemical'}, 'text': 'sphingolipid', 'locations': [{'offset': 53, 'length': 12}]}], 'relations': []}, {'infons': {'type': 'abstract', 'section': 'Abstract'}, 'offset': 127, 'text': 'Amyotrophic lateral sclerosis (ALS) is a progressive neurodeg

In [9]:
res_json

[{'id': '2',
  'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'},
  'text': 'SPTLC1',
  'locations': [{'offset': 0, 'length': 6}],
  'Pmid': ['35900868']},
 {'id': '3',
  'infons': {'identifier': 'MESH:D013107', 'type': 'Chemical'},
  'text': 'sphingolipid',
  'locations': [{'offset': 53, 'length': 12}],
  'Pmid': ['35900868']},
 {'id': '31',
  'infons': {'identifier': 'MESH:D000690', 'type': 'Disease'},
  'text': 'Amyotrophic lateral sclerosis',
  'locations': [{'offset': 127, 'length': 29}],
  'Pmid': ['35900868']},
 {'id': '32',
  'infons': {'identifier': 'MESH:D019636', 'type': 'Disease'},
  'text': 'neurodegenerative disease',
  'locations': [{'offset': 180, 'length': 25}],
  'Pmid': ['35900868']},
 {'id': '33',
  'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'},
  'text': 'SPTLC1',
  'locations': [{'offset': 248, 'length': 6}],
  'Pmid': ['35900868']},
 {'id': '34',
  'infons': {'identifier': '189', 'type': 'Gene', 'ncbi_homo

In [10]:
#print([res_json['cars'][i]['model'] for i in range(len(x['cars']))])
f = open('Pubtator_entities.txt', mode='w')

[f.write(res_json[i]['text'] + '\n') for i in range(len(res_json)) if(res_json[i]['infons']['type'] == 'Gene' or res_json[i]['infons']['type'] == 'Chemical')]
f.close()
f = open('Pubtator_entities.txt', mode='r')
print(f.read())
'''
for i in range(len(res_json)):
  if(res_json[i]['infons']['type'] == 'Gene' or res_json[i]['infons']['type'] == 'Chemical'):
    print(res_json[i]['text'], '\n')
'''


SPTLC1
sphingolipid
SPTLC1
serine-palmitoyltransferase
SPT
sphingolipids
SPTLC1
SPT
SPTLC1
sphingolipid
lipid
SPTLC1
HSAN1
SPT
L-alanine
serine
serine
SPTLC1
HSAN1
SPTLC1
HSAN1
1-deoxySL
serine
SPTLC1



"\nfor i in range(len(res_json)):\n  if(res_json[i]['infons']['type'] == 'Gene' or res_json[i]['infons']['type'] == 'Chemical'):\n    print(res_json[i]['text'], '\n')\n"

In [16]:
input_text = 'Amyotrophic lateral sclerosis (ALS) is a progressive neurodegenerative disease that affects motor neurons. Mutations in the SPTLC1 subunit of serine palmitoyltransferase (SPT), which catalyzes the first step in the de novo synthesis of sphingolipids (SLs), cause childhood-onset ALS. SPTLC1-ALS variants map to a transmembrane domain that interacts with ORMDL proteins, negative regulators of SPT activity. We show that ORMDL binding to the holoenzyme complex is impaired in cells expressing pathogenic SPTLC1-ALS alleles, resulting in increased SL synthesis and a distinct lipid signature. C-terminal SPTLC1 variants cause peripheral hereditary sensory and autonomic neuropathy type 1 (HSAN1) due to the synthesis of 1-deoxysphingolipids (1-deoxySLs) that form when SPT metabolizes L-alanine instead of L-serine. Limiting L-serine availability in SPTLC1-ALS-expressing cells increased 1-deoxySL and shifted the SL profile from an ALS to an HSAN1-like signature. This effect was corroborated in an SPTLC1-ALS pedigree in which the index patient uniquely presented with an HSAN1 phenotype, increased 1-deoxySL levels, and an L-serine deficiency. These data demonstrate how pathogenic variants in different domains of SPTLC1 give rise to distinct clinical presentations that are nonetheless modifiable by substrate availability.'

In [19]:
nlp = spacy.load("en_core_sci_sm") #instantiate scispacy model
doc = nlp(input_text)
print(doc.ents) # show entities created by the model
#for value, rel_dict in doc._.rel.items():
#    print(f"{value}: {rel_dict}")

(Amyotrophic lateral sclerosis, ALS, progressive, neurodegenerative disease, motor neurons, Mutations, SPTLC1 subunit, serine palmitoyltransferase, SPT, de novo, synthesis, sphingolipids, SLs, childhood-onset, ALS, SPTLC1-ALS, variants, map, transmembrane domain, ORMDL proteins, negative, regulators, SPT, activity, ORMDL, binding, holoenzyme complex, impaired, cells, expressing, pathogenic, SPTLC1-ALS, alleles, increased, SL, synthesis, lipid signature, C-terminal, SPTLC1, variants, peripheral, hereditary sensory, autonomic neuropathy type 1, HSAN1, synthesis, 1-deoxysphingolipids, SPT, L-alanine, L-serine, Limiting, L-serine, availability, SPTLC1-ALS-expressing cells, increased, 1-deoxySL, shifted, SL, ALS, HSAN1-like signature, effect, SPTLC1-ALS, pedigree, patient, HSAN1, phenotype, increased, 1-deoxySL levels, L-serine deficiency, data, pathogenic, variants, domains, SPTLC1, clinical presentations, substrate, availability)
